In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

In [3]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [4]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [5]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
df_spec_train = pd.read_pickle('../feats/spec_features.pkl')
df_spec_test = pd.read_pickle('../feats/spec_features_test.pkl')

In [7]:
df_train = pd.merge(df_train, df_spec_train, on='index')
df_test = pd.merge(df_test, df_spec_test, on='index')

In [8]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [9]:
len(tsfresh_columns)

9731

In [10]:
db = DFDB('../trial/catboost.pkl', auto_commit=False)

In [11]:
lgbm_top200 = ['spkt_welch_densitycoeff_2',
 'spkt_welch_density__coeff_3',
 'q25_roll_std_100',
 '3th_peak_freq',
 'min_roll_std_100',
 'q05_roll_std_100',
 'iqr_6',
 'abs_max_8',
 'mfcc_5_abs_mean',
 "number_peaks{'n': 10}",
 'ave10_7',
 'mfcc_13_mean',
 "autocorrelation{'lag': 5}",
 'mfcc_accelerate_8_variance',
 'abs_max_7',
 'mfcc_accelerate_1_kurtosis',
 'q05_roll_std_1000',
 'spkt_welch_density__coeff_42',
 'max_to_min_diff_5',
 'mfcc_13_quantile25',
 'fft_coefficientcoeff_80__attr_"imag"',
 'abs_q25_5',
 'mfcc_5_mean',
 'median__roll_std',
 '5000skewness_max_',
 'fft_coefficientcoeff_6__attr_"abs"',
 'partial_autocorrelationlag_5',
 'abs_min_8',
 'spkt_welch_density__coeff_28',
 'ar_coefficientk_10__coeff_3',
 'abs_q75_7',
 'mfcc_accelerate_15_min',
 'abs_max_4',
 'mfcc_10_quantile25',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'spkt_welch_density__coeff_79',
 "value_count{'value': 1}",
 '5000peak_peak_amp_max_',
 'min__roll_std',
 'mfcc_4_rolling_std_mean',
 'q01_roll_std_1000',
 'mfcc_3_abs_q75',
 'mfcc_5_quantile95',
 'fft_coefficientcoeff_16__attr_"imag"',
 'abs_q01_4',
 'mfcc_6_quantile05',
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 '5000quantile25skewness_',
 'mfcc_0_abs_q25',
 'mfcc_delta_3_quantile01',
 'spkt_welch_density__coeff_99',
 'mfcc_9_mean',
 'mfcc_12_mean',
 'q75_roll_mean_10',
 'max_to_min',
 'mfcc_3_abs_mean',
 'kurt_7',
 'mfcc_14_quantile01',
 'mfcc_10_mean',
 'mfcc_delta_13_min',
 'mfcc_delta_13_abs_max',
 '5000no_zero_crossing_mean_',
 'abs_q75_6',
 'mfcc_12_quantile99',
 'mfcc_accelerate_6_skewness',
 'mfcc_5_max',
 '5000smoothness_entropy_',
 '5000median_skewness_',
 'mfcc_accelerate_8_min',
 'abs_min_3',
 '5000quantile75mean_',
 "number_crossing_m{'m': 1}",
 '5000smoothness_std_',
 'mfcc_9_abs_q25',
 'flac3_1_quantile99',
 'mfcc_5_skewness',
 'fft_coefficientcoeff_56__attr_"angle"',
 'fft_coefficientcoeff_70__attr_"abs"',
 'mfcc_10_abs_q75',
 'fft_coefficientcoeff_24__attr_"angle"',
 'med_7',
 'spkt_welch_density__coeff_73',
 'abs_q99_8',
 'ave10_6',
 'spkt_welch_density__coeff_38',
 'skew_1',
 'mfcc_delta_3_abs_q95',
 "change_quantiles{'ql': 0.6, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}",
 'max_to_min_5',
 'mfcc_delta_4_quantile75',
 'abs_max_1',
 '5000crest_factor_quantile75',
 'partial_autocorrelationlag_1',
 'mfcc_delta_11_quantile01',
 'mfcc_accelerate_7_quantile01',
 'mfcc_8_rolling_std_mean',
 'q95_roll_mean_100',
 'mfcc_delta_6_rolling_std_mean',
 'flac3_0_min',
 'fft_coefficientcoeff_36__attr_"abs"',
 'iqr_8',
 'max_9',
 '5000smoothness_quantile05',
 'mfcc_1_kurtosis',
 'mfcc_7_abs_q95',
 'spkt_welch_density__coeff_66',
 'spkt_welch_density__coeff_64',
 'mfcc_3_abs_q95',
 '5000skewness_mean_',
 'mfcc_10_abs_mean',
 '5000quantile99quantile01',
 'mean_change_rate',
 'mfcc_accelerate_5_rolling_std_mean',
 'mfcc_accelerate_5_quantile05',
 '5000rms_median_',
 'flac3_1_abs_mean',
 '5000rms_quantile05',
 '5000quantile75quantile25',
 'mfcc_accelerate_12_min',
 "value_count{'value': -1}",
 'mfcc_12_abs_max',
 'abs_min_5',
 'mfcc_5_abs_q99',
 'mfcc_delta_5_quantile01',
 'mfcc_12_quantile75',
 'fft_coefficientcoeff_56__attr_"imag"',
 'spkt_welch_densitycoeff_5',
 'mfcc_0_abs_q05',
 'mfcc_13_quantile75',
 'mfcc_delta_5_min',
 'mfcc_5_quantile99',
 'fft_coefficientcoeff_8__attr_"angle"',
 'spkt_welch_density__coeff_30',
 'mfcc_accelerate_4_max',
 'mfcc_14_median',
 "change_quantiles{'ql': 0.2, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'}",
 'mfcc_5_median',
 'mfcc_accelerate_5_abs_q75',
 'spkt_welch_density__coeff_59',
 'mfcc_accelerate_5_kurtosis',
 'mfcc_delta_9_quantile99',
 'mfcc_5_quantile25',
 'spkt_welch_density__coeff_113',
 'mfcc_2_quantile05',
 'spkt_welch_density__coeff_58',
 'mfcc_5_abs_q75',
 'spkt_welch_density__coeff_22',
 'spkt_welch_density__coeff_115',
 'spkt_welch_density__coeff_4',
 'mfcc_5_abs_q05',
 'spkt_welch_density__coeff_25',
 'mfcc_5_abs_max',
 'spkt_welch_density__coeff_27',
 'mfcc_4_quantile75',
 'mfcc_3_rolling_std_mean',
 'mfcc_4_median',
 'mfcc_0_quantile01',
 '5000quantile75rssq_',
 'fft_coefficientcoeff_24__attr_"imag"',
 'mfcc_delta_12_kurtosis',
 '5000rms_quantile25',
 'mfcc_13_quantile05',
 'fft_coefficientcoeff_8__attr_"imag"',
 'mfcc_delta_11_rolling_std_mean',
 'mfcc_delta_12_quantile99',
 'mfcc_accelerate_9_abs_q25',
 'mfcc_11_abs_mean',
 'mfcc_delta_15_abs_max',
 'abs_max_2',
 'mfcc_11_abs_std',
 'abs_max_roll_mean_1000',
 'peak_to_average_power_ratio__roll_mean',
 'mfcc_13_median',
 'fft_coefficientcoeff_62__attr_"abs"',
 'mfcc_11_quantile75',
 "quantile{'q': 0.8}",
 'mfcc_delta_3_rolling_std_mean',
 'abs_q99_7',
 'mfcc_delta_12_max',
 'mfcc_12_min',
 'mfcc_10_quantile99',
 'mfcc_14_abs_q99',
 'mfcc_14_quantile05',
 'mfcc_14_quantile25',
 'kurt_1',
 'mfcc_2_median',
 'q01_2',
 'mfcc_2_abs_q25',
 'kurt_8',
 'mfcc_accelerate_6_abs_max',
 'mfcc_1_quantile75',
 'q05_5',
 'abs_q95_9',
 'q05_roll_mean_100',
 'mfcc_10_abs_q25',
 'q75_9',
 'mfcc_15_mean',
 "change_quantiles{'ql': 0.6, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'}",
 'flac3_0_abs_std',
 "change_quantiles{'ql': 0.4, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}"]

In [12]:
df_trial = db.select()
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])

In [13]:
param = {'algorithm': {'cls': 'cb.CatBoostRegressor',
  'fit': {'early_stopping_rounds': 200, 'verbose': False},
  'init': {}},
 'columns': lgbm_top200,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init':{}}}

In [17]:
mytrial =[]
EP.select_features_(df_train, param, mytrial, nfeats_best=20, nfeats_removed_per_try=10, key='average_model_weight', remark='rfe to 20 group3')

In [23]:
for trial_i in mytrial:
    db.insert(trial_i)

In [24]:
df_trial = db.select()

In [25]:
df_trial[(df_trial['remark']=='rfe to 20 group3')][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1669,2019-05-31 22:54:36.547877,rfe to 20 group3,200,1.440795,0.000410,2.079172,0.007055,0.638377
1670,2019-05-31 23:14:48.078188,rfe to 20 group3,190,1.441548,0.000512,2.081663,0.006587,0.640115
1671,2019-05-31 23:41:10.345734,rfe to 20 group3,180,1.441061,0.000384,2.078908,0.005497,0.637847
1672,2019-06-01 00:08:45.745847,rfe to 20 group3,170,1.439969,0.000412,2.080503,0.005896,0.640534
1673,2019-06-01 00:31:39.053972,rfe to 20 group3,160,1.443598,0.000400,2.082733,0.006097,0.639135
1674,2019-06-01 00:52:07.794162,rfe to 20 group3,150,1.439949,0.000428,2.083217,0.006135,0.643267
1675,2019-06-01 01:12:57.279064,rfe to 20 group3,140,1.439547,0.000370,2.084357,0.006205,0.644809
1676,2019-06-01 01:31:04.922329,rfe to 20 group3,130,1.440942,0.000526,2.086788,0.006322,0.645846
1677,2019-06-01 01:47:06.812239,rfe to 20 group3,120,1.443094,0.000421,2.088500,0.005703,0.645406
1678,2019-06-01 02:01:14.767515,rfe to 20 group3,110,1.445715,0.000429,2.087185,0.005690,0.641470


In [26]:
db.commit()

In [27]:
score = df_trial.loc[1705].val_mae

In [28]:
param = df_trial.loc[1705]['param']

In [29]:
def width_frist_rfe(df_train, param, trial, score, df_test=None, remark=None):

    param_ = copy.deepcopy(param)
    columns_ = param_['columns']
    best_score = score
    best_param = param_
    for col in columns_:
        param_['columns'] = list(set(columns_) - set([col]))
        df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param_, df_test=df_test, trial=trial, is_output_feature_importance=False, remark=remark)
        val_mae_mean = np.mean(df_his.valid)
        if val_mae_mean<best_score:
            best_score = val_mae_mean
            best_param = copy.deepcopy(param_)

    if best_score < score:
        width_frist_rfe(df_train, best_param, trial, best_score, df_test, remark=remark)

    return

In [30]:
mytrial = []
width_frist_rfe(df_train, param, mytrial, score, df_test=df_test, remark='wf 1705')

In [33]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [34]:
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])
df_trial['algorithm-init'] = df_trial['param'].apply(lambda x: x['algorithm']['init'])

In [35]:
df_trial[(df_trial['remark']=='wf 1705')].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','kfold','algorithm-init']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,kfold,algorithm-init
1861,2019-06-01 05:49:01.973133,24,1.587584,0.001020,2.043570,0.009131,0.455986,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1873,2019-06-01 05:55:57.675589,23,1.602037,0.001103,2.043956,0.009230,0.441919,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1893,2019-06-01 06:09:41.178179,23,1.607688,0.000928,2.044033,0.010119,0.436345,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1892,2019-06-01 06:08:48.549643,23,1.586726,0.000996,2.044739,0.008714,0.458013,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1874,2019-06-01 05:56:28.333721,23,1.589004,0.001151,2.046466,0.008606,0.457462,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1869,2019-06-01 05:53:16.743137,24,1.591263,0.001283,2.046472,0.009494,0.455208,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1888,2019-06-01 06:05:15.592261,23,1.584594,0.001033,2.046671,0.009420,0.462077,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1831,2019-06-01 05:33:13.963424,25,1.575130,0.001063,2.047906,0.008218,0.472776,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1887,2019-06-01 06:04:20.959700,23,1.599487,0.001200,2.047918,0.009149,0.448430,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}
1866,2019-06-01 05:51:40.987057,24,1.583996,0.001123,2.047985,0.007914,0.463989,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",{}


In [36]:
db.commit()

In [46]:
mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 4)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.0001, 1)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':copy.deepcopy(df_trial.loc[1861]['param']['columns']),
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
                'logging_level': 'Silent'
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 1861 s3')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-06-01 10:56:22,897] Finished a trial resulted in value: 1.8572906654530907. Current best value is 1.8572906654530907 with parameters: {'num_trees': 826, 'depth': 2, 'learning_rate': 0.29364602559573827, 'l2_leaf_reg': 0.9714608744655773, 'bagging_temperature': 0.6932428651879207, 'random_strength': 0.1328298190582585, 'random_state': 9870}.
[I 2019-06-01 10:56:45,096] Finished a trial resulted in value: 1.8198125648943269. Current best value is 1.8198125648943269 with parameters: {'num_trees': 483, 'depth': 3, 'learning_rate': 0.24092446731180053, 'l2_leaf_reg': 0.5703807761972379, 'bagging_temperature': 0.8357740028901302, 'random_strength': 0.5528860873205879, 'random_state': 6377}.
[I 2019-06-01 10:57:36,330] Finished a trial resulted in value: 1.6349264196117999. Current best value is 1.6349264196117999 with parameters: {'num_trees': 947, 'depth': 4, 'learning_rate': 0.35606983603738307, 'l2_leaf_reg': 0.9987857062718997, 'bagging_temperature': 0.8708650009653616, 'random_s

[I 2019-06-01 11:24:25,492] Finished a trial resulted in value: 1.7557645423717885. Current best value is 1.5773256976602807 with parameters: {'num_trees': 989, 'depth': 4, 'learning_rate': 0.35934279661571344, 'l2_leaf_reg': 0.9038544255118843, 'bagging_temperature': 0.6492967128183273, 'random_strength': 0.20421642616315105, 'random_state': 4221}.
[I 2019-06-01 11:24:44,030] Finished a trial resulted in value: 1.798325124588315. Current best value is 1.5773256976602807 with parameters: {'num_trees': 989, 'depth': 4, 'learning_rate': 0.35934279661571344, 'l2_leaf_reg': 0.9038544255118843, 'bagging_temperature': 0.6492967128183273, 'random_strength': 0.20421642616315105, 'random_state': 4221}.
[I 2019-06-01 11:25:15,741] Finished a trial resulted in value: 1.8641943149324547. Current best value is 1.5773256976602807 with parameters: {'num_trees': 989, 'depth': 4, 'learning_rate': 0.35934279661571344, 'l2_leaf_reg': 0.9038544255118843, 'bagging_temperature': 0.6492967128183273, 'random_

[I 2019-06-01 11:54:39,063] Finished a trial resulted in value: 1.7852904801908949. Current best value is 1.5762360452522846 with parameters: {'num_trees': 998, 'depth': 4, 'learning_rate': 0.37548375447054566, 'l2_leaf_reg': 0.8472566815448193, 'bagging_temperature': 0.6823749887039545, 'random_strength': 0.10114193529997645, 'random_state': 5558}.
[I 2019-06-01 11:55:09,382] Finished a trial resulted in value: 1.741477620525224. Current best value is 1.5762360452522846 with parameters: {'num_trees': 998, 'depth': 4, 'learning_rate': 0.37548375447054566, 'l2_leaf_reg': 0.8472566815448193, 'bagging_temperature': 0.6823749887039545, 'random_strength': 0.10114193529997645, 'random_state': 5558}.
[I 2019-06-01 11:55:54,601] Finished a trial resulted in value: 1.709631807478517. Current best value is 1.5762360452522846 with parameters: {'num_trees': 998, 'depth': 4, 'learning_rate': 0.37548375447054566, 'l2_leaf_reg': 0.8472566815448193, 'bagging_temperature': 0.6823749887039545, 'random_s

[I 2019-06-01 12:23:58,710] Finished a trial resulted in value: 1.7332060378135574. Current best value is 1.5724498624306327 with parameters: {'num_trees': 973, 'depth': 4, 'learning_rate': 0.39821045838391245, 'l2_leaf_reg': 0.541259073896572, 'bagging_temperature': 0.6016474620739699, 'random_strength': 0.7889405758403696, 'random_state': 489}.
[I 2019-06-01 12:24:26,598] Finished a trial resulted in value: 1.8666999728133262. Current best value is 1.5724498624306327 with parameters: {'num_trees': 973, 'depth': 4, 'learning_rate': 0.39821045838391245, 'l2_leaf_reg': 0.541259073896572, 'bagging_temperature': 0.6016474620739699, 'random_strength': 0.7889405758403696, 'random_state': 489}.
[I 2019-06-01 12:25:08,976] Finished a trial resulted in value: 1.620998256274489. Current best value is 1.5724498624306327 with parameters: {'num_trees': 973, 'depth': 4, 'learning_rate': 0.39821045838391245, 'l2_leaf_reg': 0.541259073896572, 'bagging_temperature': 0.6016474620739699, 'random_strengt

[I 2019-06-01 12:59:35,013] Finished a trial resulted in value: 1.5944741187537368. Current best value is 1.5596964007591805 with parameters: {'num_trees': 999, 'depth': 4, 'learning_rate': 0.37625188257658976, 'l2_leaf_reg': 0.8761322591481053, 'bagging_temperature': 0.6298407544387783, 'random_strength': 0.11962936276388275, 'random_state': 4687}.
[I 2019-06-01 13:00:24,797] Finished a trial resulted in value: 1.614944380008544. Current best value is 1.5596964007591805 with parameters: {'num_trees': 999, 'depth': 4, 'learning_rate': 0.37625188257658976, 'l2_leaf_reg': 0.8761322591481053, 'bagging_temperature': 0.6298407544387783, 'random_strength': 0.11962936276388275, 'random_state': 4687}.
[I 2019-06-01 13:01:03,167] Finished a trial resulted in value: 1.7528309563105964. Current best value is 1.5596964007591805 with parameters: {'num_trees': 999, 'depth': 4, 'learning_rate': 0.37625188257658976, 'l2_leaf_reg': 0.8761322591481053, 'bagging_temperature': 0.6298407544387783, 'random_

In [47]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])
df_trial['algorithm-init'] = df_trial['param'].apply(lambda x: x['algorithm']['init'])

In [49]:
df_trial[(df_trial['remark']=='tune 1861 s3')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','kfold', 'algorithm-init', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head()

,datetime,kfold,algorithm-init,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
2309,2019-06-01 11:03:15.229312,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}","{'num_trees': 533, 'depth': 4, 'learning_rate': 0.03365891519851319, 'l2_leaf_reg': 0.026391109783800604, 'bagging_temperature': 0.7676450654917518, 'random_strength': 0.7534136561493406, 'random_state': 2395, 'logging_level': 'Silent'}",24,1.885771,0.000041,1.927192,0.000063,0.041421
2423,2019-06-01 12:16:09.193683,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}","{'num_trees': 760, 'depth': 2, 'learning_rate': 0.07752152016091646, 'l2_leaf_reg': 0.03929425440040579, 'bagging_temperature': 0.7542520270530777, 'random_strength': 0.22245148118173194, 'random_state': 3276, 'logging_level': 'Silent'}",24,1.888978,0.000053,1.931460,0.000079,0.042482
2352,2019-06-01 11:29:53.809059,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}","{'num_trees': 621, 'depth': 3, 'learning_rate': 0.04268870979553088, 'l2_leaf_reg': 0.37515191846716833, 'bagging_temperature': 0.8243471685304335, 'random_strength': 0.3995922209855491, 'random_state': 172, 'logging_level': 'Silent'}",24,1.892319,0.000013,1.932635,0.000098,0.040316
2468,2019-06-01 12:50:22.939801,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}","{'num_trees': 854, 'depth': 3, 'learning_rate': 0.022355724285015893, 'l2_leaf_reg': 0.5241103791194218, 'bagging_temperature': 0.651425821035841, 'random_strength': 0.07913756023234553, 'random_state': 2973, 'logging_level': 'Silent'}",24,1.911720,0.000029,1.947568,0.000098,0.035848
2384,2019-06-01 11:52:03.339025,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}","{'num_trees': 798, 'depth': 2, 'learning_rate': 0.025367318583593257, 'l2_leaf_reg': 0.8709214745782132, 'bagging_temperature': 0.6744339811714879, 'random_strength': 0.10218776207275904, 'random_state': 6672, 'logging_level': 'Silent'}",24,1.963999,0.000028,1.985231,0.000081,0.021232


In [50]:
db.commit()